<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#notes-and-questions" data-toc-modified-id="notes-and-questions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>notes and questions</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---working-folder-paths" data-toc-modified-id="Setup---working-folder-paths-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - working folder paths</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li><li><span><a href="#Setup---ExportToContext-instance" data-toc-modified-id="Setup---ExportToContext-instance-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Setup - ExportToContext instance</a></span></li><li><span><a href="#Setup---Initialize-LoggingHelper" data-toc-modified-id="Setup---Initialize-LoggingHelper-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Setup - Initialize LoggingHelper</a></span></li></ul></li><li><span><a href="#load-sourcenet-data-into-context" data-toc-modified-id="load-sourcenet-data-into-context-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>load sourcenet data into context</a></span><ul class="toc-item"><li><span><a href="#Retrieve-Article-instances" data-toc-modified-id="Retrieve-Article-instances-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Retrieve Article instances</a></span></li><li><span><a href="#build-load-code-and-unit-tests" data-toc-modified-id="build-load-code-and-unit-tests-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>build load code and unit tests</a></span><ul class="toc-item"><li><span><a href="#Testing" data-toc-modified-id="Testing-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Testing</a></span></li></ul></li><li><span><a href="#Load-data-from-Article-instances" data-toc-modified-id="Load-data-from-Article-instances-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load data from Article instances</a></span></li></ul></li></ul></div>

# notes and questions

- Back to [Table of Contents](#Table-of-Contents)

Notes:

- probably will need a class that is an Article_Data container - reference to Article_Data, and then all the logic to process the entities and relations contained within.

Questions:

- Do we want an Identifier Type separate from Entity and Relation identifiers?  I think we do, so we can specify the entity type(s) a given identifier should be used on.

    - Created abstract one, and created a concrete entity identifier type.  Will create one for relation identifiers if needed.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
me = "sourcenet-to-context"

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [2]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [15]:
import datetime
from django.db.models import Avg, Max, Min
from django.utils.text import slugify
import gc
import json
import logging
import six

## Setup - working folder paths

- Back to [Table of Contents](#Table-of-Contents)

In [4]:
%pwd

'/home/jonathanmorgan/work/django/research/work/phd_work/analysis'

In [5]:
# current working folder
current_working_folder = "/home/jonathanmorgan/work/django/research/work/phd_work/analysis"
current_datetime = datetime.datetime.now()
current_date_string = current_datetime.strftime( "%Y-%m-%d-%H-%M-%S" )

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [6]:
logging_file_name = "{}/{}-{}.log.txt".format( current_working_folder, me, current_date_string )
logging.basicConfig(
    level = logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = logging_file_name,
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)
print( "Logging initialized, to {}".format( logging_file_name ) )

Logging initialized, to /home/jonathanmorgan/work/django/research/work/phd_work/analysis/sourcenet-to-context-2019-11-29-01-56-55.log.txt


## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [7]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [8]:
%run $django_init_path

django initialized at 2019-11-29 01:57:00.635825


In [9]:
# context imports
from context.models import Entity
from context.models import Entity_Identifier_Type
from context.models import Entity_Identifier
from context.models import Entity_Type

# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.export.to_context_base.export_to_context import ExportToContext
from context_text.models import Article
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase

## Setup - ExportToContext instance

- Back to [Table of Contents](#Table-of-Contents)

Make instance, set instance variables.

In [10]:
my_exporter = ExportToContext()

# no variables to set, yet...
my_exporter.set_article_uuid_id_type_name( ExportToContext.ENTITY_ID_TYPE_ARTICLE_NEWSBANK_ID )

<Entity_Identifier_Type: 4 - article_newsbank_id - Newsbank - ( article )>

## Setup - Initialize LoggingHelper

- Back to [Table of Contents](#Table-of-Contents)

Create a LoggingHelper instance to use to log debug and also print at the same time.

Preconditions: Must be run after Django is initialized, since `python_utilities` is in the django path.

In [11]:
# python_utilities
from python_utilities.logging.logging_helper import LoggingHelper

# init
my_logging_helper = LoggingHelper()
my_logging_helper.set_logger_name( me )
log_message = None

# load sourcenet data into context

- Back to [Table of Contents](#Table-of-Contents)

logic:

- get all Article_Data created by automated coder, open calais coder type.
- for each:

    - Article processing:
    
        - look for article entity in context.  If not created, create one of type "article".  Store the sourcenet ID as identifier of type "article_sourcenet_id".
        - Store the ID and entity reference for the Article (in object?  Should I make a place for the article to hold an Entity ID?).
        - check to see if the newspaper has an entity (using its sourcenet ID as the lookup identifier).  If not, create one.

    - Person processing:
    
        - Add all people who are either reporters/authors or subjects to `context` as entities of type "person", with identifier of type `person_sourcenet_id` set to their internal django/database "id", and with identifier of type `person_open_calais_uuid` set to their OpenCalais ID, if they have one.  If they have any other types of IDs, add them too, untyped.  Once entity is created, store ForeignKey of entity in Person record.
        - Author/Reporter processing:

            - look for each author's person entity.  If not found, create one.
    
        - Subject/Source processing:
    
            - look for a person entity for each subject/source.  If not found, create one.

    - Relations - create the following:
    
        - from newspaper
        
            - newspaper_reporter - Reporter at a newspaper, evidence of which is byline on an article in that newspaper. FROM newspaper TO person (reporter) THROUGH article.
            - newspaper_source - Person quoted in an article published by a newspaper. FROM newspaper TO person (source) THROUGH article.
            - newspaper_subject - Subject of an article published in a given newspaper. FROM newspaper TO person (subject, including sources) THROUGH article.
            - newspaper_article - Article published in a particular newspaper. FROM newspaper TO article.

        - from article

            - author - Author/Reporter of an article - FROM article TO reporter.
            - subject - Subject of a story. FROM article TO subject person.
            - source - Source quoted in an article - FROM article TO source person.

        - through article

            - article_container - Parent for relations based on entities being mentioned in the same article. To start, just people, but eventually, for example, could also include location.
            - mentioned - Mentioned in an article. FROM reporter/author TO subject THROUGH article.
            - quoted - The "from" person quoted the "to" person in a publication. FROM reporter TO source THROUGH article.
            - same_article_sources - Sources in the same article, FROM source person TO source person THROUGH article.
            - same_article_subjects - Two people who are in a particular article together (includes subjects and sources).
            - shared_byline - Shared Byline on an article - joint authors - FROM author TO author THROUGH article.

Relations broken out by person type:

- For all reporters/authors:

    - create a relation of type "`author`" between the article's entity (FROM) and the entity of the person (TO) for each author.
    - if multiple authors, for each pair of authors, create a relation of type "`shared_byline`" between the two (it is undirected), THROUGH the article.

- For all subjects, including sources:

    - create a relation of type "`subject`" between the article's entity (FROM) and the entity of the subject person (TO).
    - if multiple subjects, for each pair of subjects, create a relation of type "`same_article_subjects`" between the two (it is undirected), THROUGH the article.
    - create a relation of type "`mentioned`" between each of the article's authors (FROM) and the subject (TO), THROUGH the article.

- For all sources

    - create a relation of type "`source`" between the article's entity and the entity for the source person.
    - if multiple sources, for each pair of sources, create a relation of type "`same_article_sources`" between the two (it is undirected), THROUGH the article.
    - create a relation of type "`quoted`" between each of the article's authors (FROM) and the source (TO), THROUGH the article.


Convenience methods:

- method to find entity - based on type and identifier (accept all the fields that make sense, including optional identifier type instance).
- method to find relation - based on type, etc.

Notes:

- all ties are undirected.
- relations can have three foreign keys into Entity - FROM, TO, and THROUGH (for a containing relationship, like the article that included two sources that we are relating).

## Retrieve Article instances

- Back to [Table of Contents](#Table-of-Contents)

Before we do anything else, need to be able to pull back all the articles whose data we want to load into the context store.

In [12]:
# look for publications that have article data:
# - coded by automated coder
# - with coder type of "OpenCalais_REST_API_v2"

# get automated coder
automated_coder_user = ContextTextBase.get_automated_coding_user()

log_message = "{} - Loaded automated user: {}, id = {}".format( datetime.datetime.now(), automated_coder_user, automated_coder_user.id )
my_logging_helper.output_message( log_message, do_print_IN = True, log_level_code_IN = logging.INFO )

2019-11-29 01:57:07.200553 - Loaded automated user: automated, id = 2


In [13]:
# find articles with Article_Data created by the automated user...
article_qs = Article.objects.filter( article_data__coder = automated_coder_user )

# ...and specifically coded using OpenCalais V2...
article_qs = article_qs.filter( article_data__coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )

# ...and finally, we just want the distinct articles by ID.
article_qs = article_qs.order_by( "id" ).distinct( "id" )

# count?
article_count = article_qs.count()
log_message = "Found {} articles".format( article_count )
my_logging_helper.output_message( log_message, do_print_IN = True, log_level_code_IN = logging.INFO )

Found 43843 articles


## build load code and unit tests

- Back to [Table of Contents](#Table-of-Contents)

### Testing

- Back to [Table of Contents](#Table-of-Contents)

First, make unit tests to test convenience methods added to the following models, in `context/tests/models/`:

- // `Entity_Identifier_Type` - test_Entity_Identifier_Type_model.py
- // `Entity_Identifier` - test_Entity_Identifier_model.py
- // `Entity` - test_Entity_model.py

Also, // move instance creation class methods along with their constants over into "TestHelper" from test_Entity_Identifier_model.py, so they can be re-used across test classes.

To run: `python manage.py test context.tests`

In test data:

- article 21925:

    - Article_Data: `SELECT * FROM context_text_article_data WHERE article_id = 21925;`

TODO:

- test_entity_model.py

    - test_get_entity_for_identifier

- test_export_to_context.py

    - test_get_article_uuid_id_type
    - test_set_article_uuid_id_type_name

## Load data from Article instances

- Back to [Table of Contents](#Table-of-Contents)

Now, we actually load the data...

In [14]:
result = my_exporter.process_articles( article_qs )

In create_q_filter_automated_by_coder_type: automated coder user: 2 - automated
----> Processed 100 of 43843 articles at 2019-11-26 16:44:10.194920 ( elapsed: 0:02:04.258806; average: 0:00:01.242588 )
----> Processed 200 of 43843 articles at 2019-11-26 16:46:08.333038 ( elapsed: 0:04:02.396924; average: 0:00:01.211985 )
----> Processed 300 of 43843 articles at 2019-11-26 16:48:03.126190 ( elapsed: 0:05:57.190076; average: 0:00:01.190634 )
----> Processed 400 of 43843 articles at 2019-11-26 16:50:09.831284 ( elapsed: 0:08:03.895170; average: 0:00:01.209738 )
----> Processed 500 of 43843 articles at 2019-11-26 16:52:15.175648 ( elapsed: 0:10:09.239534; average: 0:00:01.218479 )
----> Processed 600 of 43843 articles at 2019-11-26 16:54:02.125858 ( elapsed: 0:11:56.189744; average: 0:00:01.193650 )
----> Processed 700 of 43843 articles at 2019-11-26 16:55:50.575779 ( elapsed: 0:13:44.639665; average: 0:00:01.178057 )
----> Processed 800 of 43843 articles at 2019-11-26 16:57:46.909539 ( ela

----> Processed 6800 of 43843 articles at 2019-11-26 20:15:53.821748 ( elapsed: 3:33:47.885634; average: 0:00:01.886454 )
----> Processed 6900 of 43843 articles at 2019-11-26 20:19:51.879550 ( elapsed: 3:37:45.943436; average: 0:00:01.893615 )
----> Processed 7000 of 43843 articles at 2019-11-26 20:23:44.463678 ( elapsed: 3:41:38.527564; average: 0:00:01.899790 )
----> Processed 7100 of 43843 articles at 2019-11-26 20:28:01.227684 ( elapsed: 3:45:55.291570; average: 0:00:01.909196 )
----> Processed 7200 of 43843 articles at 2019-11-26 20:34:57.971819 ( elapsed: 3:52:52.035705; average: 0:00:01.940561 )
----> Processed 7300 of 43843 articles at 2019-11-26 20:39:36.914506 ( elapsed: 3:57:30.978392; average: 0:00:01.952189 )
----> Processed 7400 of 43843 articles at 2019-11-26 20:44:19.426220 ( elapsed: 4:02:13.490106; average: 0:00:01.963985 )
----> Processed 7500 of 43843 articles at 2019-11-26 20:48:00.090877 ( elapsed: 4:05:54.154763; average: 0:00:01.967221 )
----> Processed 7600 of 

----> Processed 13500 of 43843 articles at 2019-11-27 00:41:51.046239 ( elapsed: 7:59:45.110125; average: 0:00:02.132230 )
----> Processed 13600 of 43843 articles at 2019-11-27 00:45:35.271025 ( elapsed: 8:03:29.334911; average: 0:00:02.133039 )
----> Processed 13700 of 43843 articles at 2019-11-27 00:49:50.928313 ( elapsed: 8:07:44.992199; average: 0:00:02.136131 )
----> Processed 13800 of 43843 articles at 2019-11-27 00:54:11.054125 ( elapsed: 8:12:05.118011; average: 0:00:02.139501 )
----> Processed 13900 of 43843 articles at 2019-11-27 00:58:14.435663 ( elapsed: 8:16:08.499549; average: 0:00:02.141619 )
----> Processed 14000 of 43843 articles at 2019-11-27 01:01:54.309626 ( elapsed: 8:19:48.373512; average: 0:00:02.142027 )
----> Processed 14100 of 43843 articles at 2019-11-27 01:04:58.391259 ( elapsed: 8:22:52.455145; average: 0:00:02.139890 )
----> Processed 14200 of 43843 articles at 2019-11-27 01:09:26.276984 ( elapsed: 8:27:20.340870; average: 0:00:02.143686 )
----> Processed 

----> Processed 20200 of 43843 articles at 2019-11-27 05:17:31.972823 ( elapsed: 12:35:26.036709; average: 0:00:02.243863 )
----> Processed 20300 of 43843 articles at 2019-11-27 05:21:59.604943 ( elapsed: 12:39:53.668829; average: 0:00:02.245994 )
----> Processed 20400 of 43843 articles at 2019-11-27 05:25:41.249011 ( elapsed: 12:43:35.312897; average: 0:00:02.245849 )
----> Processed 20500 of 43843 articles at 2019-11-27 05:29:16.673446 ( elapsed: 12:47:10.737332; average: 0:00:02.245402 )
----> Processed 20600 of 43843 articles at 2019-11-27 05:33:37.457739 ( elapsed: 12:51:31.521625; average: 0:00:02.247161 )
----> Processed 20700 of 43843 articles at 2019-11-27 05:37:33.585954 ( elapsed: 12:55:27.649840; average: 0:00:02.247713 )
----> Processed 20800 of 43843 articles at 2019-11-27 05:40:56.453835 ( elapsed: 12:58:50.517721; average: 0:00:02.246660 )
----> Processed 20900 of 43843 articles at 2019-11-27 05:45:06.855335 ( elapsed: 13:03:00.919221; average: 0:00:02.247891 )
----> Pr

----> Processed 26900 of 43843 articles at 2019-11-27 09:47:26.666687 ( elapsed: 17:05:20.730573; average: 0:00:02.287016 )
----> Processed 27000 of 43843 articles at 2019-11-27 09:50:28.504067 ( elapsed: 17:08:22.567953; average: 0:00:02.285280 )
----> Processed 27100 of 43843 articles at 2019-11-27 09:54:29.767491 ( elapsed: 17:12:23.831377; average: 0:00:02.285750 )
----> Processed 27200 of 43843 articles at 2019-11-27 09:58:17.432869 ( elapsed: 17:16:11.496755; average: 0:00:02.285717 )
----> Processed 27300 of 43843 articles at 2019-11-27 10:03:19.409677 ( elapsed: 17:21:13.473563; average: 0:00:02.288406 )
----> Processed 27400 of 43843 articles at 2019-11-27 10:06:40.888947 ( elapsed: 17:24:34.952833; average: 0:00:02.287407 )
----> Processed 27500 of 43843 articles at 2019-11-27 10:09:50.909678 ( elapsed: 17:27:44.973564; average: 0:00:02.285999 )
----> Processed 27600 of 43843 articles at 2019-11-27 10:13:56.698628 ( elapsed: 17:31:50.762514; average: 0:00:02.286622 )
----> Pr

----> Processed 33600 of 43843 articles at 2019-11-28 03:07:57.613031 ( elapsed: 1 day, 10:25:51.676917; average: 0:00:03.689038 )
----> Processed 33700 of 43843 articles at 2019-11-28 03:12:54.094498 ( elapsed: 1 day, 10:30:48.158384; average: 0:00:03.686889 )
----> Processed 33800 of 43843 articles at 2019-11-28 03:21:14.100914 ( elapsed: 1 day, 10:39:08.164800; average: 0:00:03.690774 )
----> Processed 33900 of 43843 articles at 2019-11-28 03:25:54.541502 ( elapsed: 1 day, 10:43:48.605388; average: 0:00:03.688159 )
----> Processed 34000 of 43843 articles at 2019-11-28 03:29:30.033613 ( elapsed: 1 day, 10:47:24.097499; average: 0:00:03.683650 )
----> Processed 34100 of 43843 articles at 2019-11-28 03:33:37.835307 ( elapsed: 1 day, 10:51:31.899193; average: 0:00:03.680114 )
----> Processed 34200 of 43843 articles at 2019-11-28 04:22:12.270743 ( elapsed: 1 day, 11:40:06.334629; average: 0:00:03.754571 )
----> Processed 34300 of 43843 articles at 2019-11-28 04:25:56.290346 ( elapsed: 1 

----> Processed 39900 of 43843 articles at 2019-11-28 08:05:58.954811 ( elapsed: 1 day, 15:23:53.018697; average: 0:00:03.554712 )
----> Processed 40000 of 43843 articles at 2019-11-28 08:10:05.119030 ( elapsed: 1 day, 15:27:59.182916; average: 0:00:03.551980 )
----> Processed 40100 of 43843 articles at 2019-11-28 08:13:24.173449 ( elapsed: 1 day, 15:31:18.237335; average: 0:00:03.548086 )
----> Processed 40200 of 43843 articles at 2019-11-28 08:17:30.291790 ( elapsed: 1 day, 15:35:24.355676; average: 0:00:03.545382 )
----> Processed 40300 of 43843 articles at 2019-11-28 08:21:59.255856 ( elapsed: 1 day, 15:39:53.319742; average: 0:00:03.543259 )
----> Processed 40400 of 43843 articles at 2019-11-28 08:25:48.353078 ( elapsed: 1 day, 15:43:42.416964; average: 0:00:03.540159 )
----> Processed 40500 of 43843 articles at 2019-11-28 08:31:17.893266 ( elapsed: 1 day, 15:49:11.957152; average: 0:00:03.539554 )
----> Processed 40600 of 43843 articles at 2019-11-28 08:35:12.589951 ( elapsed: 1 

In [15]:
print( "Result: {}".format( result ) )

Result: success: tags = []; messages = ['\nSummary:\n- good count: 43843\n- > 1 count: 0\n- 0 count: 0\n- unexpected count: 0']


In [14]:
# tag all of the articles with export_to_context-YYYYMMDD-HHMMSS
tag_value = ExportToContext.TAG_PREFIX + "20191126-164206"
print( "Tag value: {}".format( tag_value ) )

Tag value: export_to_context-20191126-164206


In [16]:
# declare variables
progress_interval = None
article_counter = None
current_time = None
previous_time = None
elapsed_time = None
elapsed_this_period = None
average_time = None
average_this_period = None

# init
progress_interval = 100
article_counter = 0
start_time = datetime.datetime.now()
current_time = start_time

# loop over articles
for current_article in article_qs:
    
    # increment counter
    article_counter += 1
    
    # add tag.
    current_article.tags.add( tag_value )
    
    # output an update every progress_interval articles.
    if ( ( article_counter % progress_interval ) == 0 ):

        previous_time = current_time
        current_time = datetime.datetime.now()
        elapsed_time = current_time - start_time
        elapsed_this_period = current_time - previous_time
        average_time = elapsed_time / article_counter
        average_this_period = elapsed_this_period / progress_interval
        print( "\n----> Processed {} of {} articles at {}".format( article_counter, article_count, current_time ) )
        print( "Total elapsed: {} ( average: {} )".format( elapsed_time, average_time ) )
        print( "Period elapsed: {} ( average: {} )".format( elapsed_this_period, average_this_period ) )

        # also, garbage collect
        #gc.collect()

    #-- END check to see if we output progress update. --#
    
#-- END loop over articles to tag. --#


----> Processed 100 of 43843 articles at 2019-11-29 02:02:50.939196
Total elapsed: 0:00:05.715275 ( average: 0:00:00.057153 )
Period elapsed: 0:00:05.715275 ( average: 0:00:00.057153 )

----> Processed 200 of 43843 articles at 2019-11-29 02:02:51.433714
Total elapsed: 0:00:06.209793 ( average: 0:00:00.031049 )
Period elapsed: 0:00:00.494518 ( average: 0:00:00.004945 )

----> Processed 300 of 43843 articles at 2019-11-29 02:02:51.922741
Total elapsed: 0:00:06.698820 ( average: 0:00:00.022329 )
Period elapsed: 0:00:00.489027 ( average: 0:00:00.004890 )

----> Processed 400 of 43843 articles at 2019-11-29 02:02:52.409249
Total elapsed: 0:00:07.185328 ( average: 0:00:00.017963 )
Period elapsed: 0:00:00.486508 ( average: 0:00:00.004865 )

----> Processed 500 of 43843 articles at 2019-11-29 02:02:52.846413
Total elapsed: 0:00:07.622492 ( average: 0:00:00.015245 )
Period elapsed: 0:00:00.437164 ( average: 0:00:00.004372 )

----> Processed 600 of 43843 articles at 2019-11-29 02:02:53.293409
T


----> Processed 4500 of 43843 articles at 2019-11-29 02:03:11.693440
Total elapsed: 0:00:26.469519 ( average: 0:00:00.005882 )
Period elapsed: 0:00:00.483214 ( average: 0:00:00.004832 )

----> Processed 4600 of 43843 articles at 2019-11-29 02:03:12.139540
Total elapsed: 0:00:26.915619 ( average: 0:00:00.005851 )
Period elapsed: 0:00:00.446100 ( average: 0:00:00.004461 )

----> Processed 4700 of 43843 articles at 2019-11-29 02:03:12.609203
Total elapsed: 0:00:27.385282 ( average: 0:00:00.005827 )
Period elapsed: 0:00:00.469663 ( average: 0:00:00.004697 )

----> Processed 4800 of 43843 articles at 2019-11-29 02:03:13.060521
Total elapsed: 0:00:27.836600 ( average: 0:00:00.005799 )
Period elapsed: 0:00:00.451318 ( average: 0:00:00.004513 )

----> Processed 4900 of 43843 articles at 2019-11-29 02:03:13.494443
Total elapsed: 0:00:28.270522 ( average: 0:00:00.005769 )
Period elapsed: 0:00:00.433922 ( average: 0:00:00.004339 )

----> Processed 5000 of 43843 articles at 2019-11-29 02:03:13.92


----> Processed 8900 of 43843 articles at 2019-11-29 02:03:32.057823
Total elapsed: 0:00:46.833902 ( average: 0:00:00.005262 )
Period elapsed: 0:00:00.452160 ( average: 0:00:00.004522 )

----> Processed 9000 of 43843 articles at 2019-11-29 02:03:32.530643
Total elapsed: 0:00:47.306722 ( average: 0:00:00.005256 )
Period elapsed: 0:00:00.472820 ( average: 0:00:00.004728 )

----> Processed 9100 of 43843 articles at 2019-11-29 02:03:32.991859
Total elapsed: 0:00:47.767938 ( average: 0:00:00.005249 )
Period elapsed: 0:00:00.461216 ( average: 0:00:00.004612 )

----> Processed 9200 of 43843 articles at 2019-11-29 02:03:33.487994
Total elapsed: 0:00:48.264073 ( average: 0:00:00.005246 )
Period elapsed: 0:00:00.496135 ( average: 0:00:00.004961 )

----> Processed 9300 of 43843 articles at 2019-11-29 02:03:34.022083
Total elapsed: 0:00:48.798162 ( average: 0:00:00.005247 )
Period elapsed: 0:00:00.534089 ( average: 0:00:00.005341 )

----> Processed 9400 of 43843 articles at 2019-11-29 02:03:34.51


----> Processed 13300 of 43843 articles at 2019-11-29 02:03:52.179434
Total elapsed: 0:01:06.955513 ( average: 0:00:00.005034 )
Period elapsed: 0:00:00.512184 ( average: 0:00:00.005122 )

----> Processed 13400 of 43843 articles at 2019-11-29 02:03:52.679864
Total elapsed: 0:01:07.455943 ( average: 0:00:00.005034 )
Period elapsed: 0:00:00.500430 ( average: 0:00:00.005004 )

----> Processed 13500 of 43843 articles at 2019-11-29 02:03:53.142400
Total elapsed: 0:01:07.918479 ( average: 0:00:00.005031 )
Period elapsed: 0:00:00.462536 ( average: 0:00:00.004625 )

----> Processed 13600 of 43843 articles at 2019-11-29 02:03:53.633967
Total elapsed: 0:01:08.410046 ( average: 0:00:00.005030 )
Period elapsed: 0:00:00.491567 ( average: 0:00:00.004916 )

----> Processed 13700 of 43843 articles at 2019-11-29 02:03:54.097664
Total elapsed: 0:01:08.873743 ( average: 0:00:00.005027 )
Period elapsed: 0:00:00.463697 ( average: 0:00:00.004637 )

----> Processed 13800 of 43843 articles at 2019-11-29 02:03


----> Processed 17700 of 43843 articles at 2019-11-29 02:04:11.867475
Total elapsed: 0:01:26.643554 ( average: 0:00:00.004895 )
Period elapsed: 0:00:00.431937 ( average: 0:00:00.004319 )

----> Processed 17800 of 43843 articles at 2019-11-29 02:04:12.322327
Total elapsed: 0:01:27.098406 ( average: 0:00:00.004893 )
Period elapsed: 0:00:00.454852 ( average: 0:00:00.004549 )

----> Processed 17900 of 43843 articles at 2019-11-29 02:04:12.825236
Total elapsed: 0:01:27.601315 ( average: 0:00:00.004894 )
Period elapsed: 0:00:00.502909 ( average: 0:00:00.005029 )

----> Processed 18000 of 43843 articles at 2019-11-29 02:04:13.317387
Total elapsed: 0:01:28.093466 ( average: 0:00:00.004894 )
Period elapsed: 0:00:00.492151 ( average: 0:00:00.004922 )

----> Processed 18100 of 43843 articles at 2019-11-29 02:04:13.782051
Total elapsed: 0:01:28.558130 ( average: 0:00:00.004893 )
Period elapsed: 0:00:00.464664 ( average: 0:00:00.004647 )

----> Processed 18200 of 43843 articles at 2019-11-29 02:04


----> Processed 22100 of 43843 articles at 2019-11-29 02:04:32.729332
Total elapsed: 0:01:47.505411 ( average: 0:00:00.004864 )
Period elapsed: 0:00:00.583618 ( average: 0:00:00.005836 )

----> Processed 22200 of 43843 articles at 2019-11-29 02:04:33.293187
Total elapsed: 0:01:48.069266 ( average: 0:00:00.004868 )
Period elapsed: 0:00:00.563855 ( average: 0:00:00.005639 )

----> Processed 22300 of 43843 articles at 2019-11-29 02:04:33.785166
Total elapsed: 0:01:48.561245 ( average: 0:00:00.004868 )
Period elapsed: 0:00:00.491979 ( average: 0:00:00.004920 )

----> Processed 22400 of 43843 articles at 2019-11-29 02:04:34.241177
Total elapsed: 0:01:49.017256 ( average: 0:00:00.004867 )
Period elapsed: 0:00:00.456011 ( average: 0:00:00.004560 )

----> Processed 22500 of 43843 articles at 2019-11-29 02:04:34.709935
Total elapsed: 0:01:49.486014 ( average: 0:00:00.004866 )
Period elapsed: 0:00:00.468758 ( average: 0:00:00.004688 )

----> Processed 22600 of 43843 articles at 2019-11-29 02:04


----> Processed 26500 of 43843 articles at 2019-11-29 02:04:53.875870
Total elapsed: 0:02:08.651949 ( average: 0:00:00.004855 )
Period elapsed: 0:00:00.443988 ( average: 0:00:00.004440 )

----> Processed 26600 of 43843 articles at 2019-11-29 02:04:54.342012
Total elapsed: 0:02:09.118091 ( average: 0:00:00.004854 )
Period elapsed: 0:00:00.466142 ( average: 0:00:00.004661 )

----> Processed 26700 of 43843 articles at 2019-11-29 02:04:54.832760
Total elapsed: 0:02:09.608839 ( average: 0:00:00.004854 )
Period elapsed: 0:00:00.490748 ( average: 0:00:00.004907 )

----> Processed 26800 of 43843 articles at 2019-11-29 02:04:55.281627
Total elapsed: 0:02:10.057706 ( average: 0:00:00.004853 )
Period elapsed: 0:00:00.448867 ( average: 0:00:00.004489 )

----> Processed 26900 of 43843 articles at 2019-11-29 02:04:55.753305
Total elapsed: 0:02:10.529384 ( average: 0:00:00.004852 )
Period elapsed: 0:00:00.471678 ( average: 0:00:00.004717 )

----> Processed 27000 of 43843 articles at 2019-11-29 02:04


----> Processed 30900 of 43843 articles at 2019-11-29 02:05:13.783715
Total elapsed: 0:02:28.559794 ( average: 0:00:00.004808 )
Period elapsed: 0:00:00.433974 ( average: 0:00:00.004340 )

----> Processed 31000 of 43843 articles at 2019-11-29 02:05:14.205228
Total elapsed: 0:02:28.981307 ( average: 0:00:00.004806 )
Period elapsed: 0:00:00.421513 ( average: 0:00:00.004215 )

----> Processed 31100 of 43843 articles at 2019-11-29 02:05:14.633987
Total elapsed: 0:02:29.410066 ( average: 0:00:00.004804 )
Period elapsed: 0:00:00.428759 ( average: 0:00:00.004288 )

----> Processed 31200 of 43843 articles at 2019-11-29 02:05:15.063098
Total elapsed: 0:02:29.839177 ( average: 0:00:00.004803 )
Period elapsed: 0:00:00.429111 ( average: 0:00:00.004291 )

----> Processed 31300 of 43843 articles at 2019-11-29 02:05:15.496027
Total elapsed: 0:02:30.272106 ( average: 0:00:00.004801 )
Period elapsed: 0:00:00.432929 ( average: 0:00:00.004329 )

----> Processed 31400 of 43843 articles at 2019-11-29 02:05


----> Processed 35300 of 43843 articles at 2019-11-29 02:05:33.883120
Total elapsed: 0:02:48.659199 ( average: 0:00:00.004778 )
Period elapsed: 0:00:00.431746 ( average: 0:00:00.004317 )

----> Processed 35400 of 43843 articles at 2019-11-29 02:05:34.346635
Total elapsed: 0:02:49.122714 ( average: 0:00:00.004777 )
Period elapsed: 0:00:00.463515 ( average: 0:00:00.004635 )

----> Processed 35500 of 43843 articles at 2019-11-29 02:05:34.825252
Total elapsed: 0:02:49.601331 ( average: 0:00:00.004778 )
Period elapsed: 0:00:00.478617 ( average: 0:00:00.004786 )

----> Processed 35600 of 43843 articles at 2019-11-29 02:05:35.334438
Total elapsed: 0:02:50.110517 ( average: 0:00:00.004778 )
Period elapsed: 0:00:00.509186 ( average: 0:00:00.005092 )

----> Processed 35700 of 43843 articles at 2019-11-29 02:05:35.846631
Total elapsed: 0:02:50.622710 ( average: 0:00:00.004779 )
Period elapsed: 0:00:00.512193 ( average: 0:00:00.005122 )

----> Processed 35800 of 43843 articles at 2019-11-29 02:05


----> Processed 39700 of 43843 articles at 2019-11-29 02:05:54.583500
Total elapsed: 0:03:09.359579 ( average: 0:00:00.004770 )
Period elapsed: 0:00:00.471424 ( average: 0:00:00.004714 )

----> Processed 39800 of 43843 articles at 2019-11-29 02:05:55.021004
Total elapsed: 0:03:09.797083 ( average: 0:00:00.004769 )
Period elapsed: 0:00:00.437504 ( average: 0:00:00.004375 )

----> Processed 39900 of 43843 articles at 2019-11-29 02:05:55.474238
Total elapsed: 0:03:10.250317 ( average: 0:00:00.004768 )
Period elapsed: 0:00:00.453234 ( average: 0:00:00.004532 )

----> Processed 40000 of 43843 articles at 2019-11-29 02:05:56.084270
Total elapsed: 0:03:10.860349 ( average: 0:00:00.004772 )
Period elapsed: 0:00:00.610032 ( average: 0:00:00.006100 )

----> Processed 40100 of 43843 articles at 2019-11-29 02:05:56.539444
Total elapsed: 0:03:11.315523 ( average: 0:00:00.004771 )
Period elapsed: 0:00:00.455174 ( average: 0:00:00.004552 )

----> Processed 40200 of 43843 articles at 2019-11-29 02:05

In [20]:
# Count number of articles with tag name starting with
#     ExportToContext.TAG_PREFIX
tag_test_qs = Article.objects.filter( tags__name__istartswith = ExportToContext.TAG_PREFIX )
tag_test_count = tag_test_qs.count()
article_count = article_qs.count()

# did all the articles get tagged?
test_value = tag_test_count
should_be = article_count
error_string = "Count of tagged articles: {}, should be {}.".format( test_value, should_be )
assert test_value == should_be, error_string

print( "Found {} Articles with tag starting with {}.".format( tag_test_count, ExportToContext.TAG_PREFIX ) )

Found 43843 Articles with tag starting with export_to_context-.
